# Historical analysis on precipitation in Rotterdam 1974 - 2019

by: Sara Mendoza

Data Analytics - Ironhack Amsterdam / cohort Jan - June 2020

Project 4 - May 2020


## 1 Introduction

In this notebook I analyze the daily rainfall measured in Rotterdam from 1974 to 2019.

The measured rainfall data can be found here: http://projects.knmi.nl/klimatologie/daggegevens/selectie.cgi

More information on rain in the Netherlands quoted throughout this notebook can be found here: http://bibliotheek.knmi.nl/weerbrochures/FS_Regen.pdf

The file used has been stored in google drive, and can be downloaded from here: https://drive.google.com/drive/folders/1dhg49rF5hWF0_fBiGRtK_2niaw5Od5cA


### Target

Climate change is affecting rain patterns all over the world. In this notebook I try to analyze how the rain has been changing in Rotterdam over the last 50 years.


## 2 Importing libraries and reading the files

In [ ]:
import pandas as pd
import numpy as np
import math
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
# reading the files

station = pd.read_csv('../data/knmi_stations_2018.csv',sep=';')
rain = pd.read_excel('../data/KNMI_20200427.xlsx')

# station.head()
# rain.head()

## 2 Selecting and cleaning the data
Before I start analyzing the data, I need to select the information that I will use, change some of the data types and check for strange values

In [ ]:
# cleaning blank spaces in columns and station names:
station.columns = station.columns.str.replace(" ","")
station['NAME'] = station['NAME'].str.replace(" ","")
rain.columns = rain.columns.str.replace(" ","")

# change Date data type to DATE
rain['YYYYMMDD'] = pd.to_datetime(rain['YYYYMMDD'].astype(int), format='%Y%m%d')

station.dtypes #all correct
rain.dtypes #all correct

# checking for blank values:
null_cols = station.isna().sum()
null_cols[null_cols > 0] # no blanks

#identifying RTM station CODE:
rtmstation = station[station['NAME'].isin(['ROTTERDAM'])]['STN']

#identifying rain information from RTM station
rtmrain = rain[(rain['#STN'] == 344)]

# checking for blank values in RTM:
null_cols = rtmrain.isna().sum()
null_cols[null_cols > 0] # has plenty of blanks, but I'll focus on certain columns only

# selecting relevant columns for precipitation analysis
# YYYYMMDD - Date
# DR - duration of precipitation (in 0.1 hours)
# RH - sum of precipitation for one day 
# RHX - highest recorded precipitation in one hour 

rtmrain = rtmrain[['YYYYMMDD','DR','RH','RHX']].reset_index()
rtmrain.drop('index',1,inplace=True)

# adding columns for month, year, semester, decade, etc. for plotting purposes 
rtmrain['month'] = pd.DatetimeIndex(rtmrain['YYYYMMDD']).month
rtmrain['year'] = pd.DatetimeIndex(rtmrain['YYYYMMDD']).year
rtmrain['quarter']= rtmrain['month'].apply(lambda x: math.floor((x-1)/3))
rtmrain['semester']= rtmrain['month'].apply(lambda x: math.floor((x-1)/6))
rtmrain['half_decade'] = (pd.DatetimeIndex(rtmrain['YYYYMMDD']).year)/5
rtmrain['half_decade']= rtmrain['half_decade'].apply(lambda x: math.floor(x))*5
rtmrain['decade'] = (pd.DatetimeIndex(rtmrain['YYYYMMDD']).year)/10
rtmrain['decade']= rtmrain['decade'].apply(lambda x: math.floor(x))*10

# checking for blank values in rtmrain:
null_cols = rtmrain.isna().sum()
null_cols[null_cols > 0] # has plenty of blanks

# it seems they only started measuring rainfall in 1974, so I will drop everything before that
rtmrain[6290:6315]

# Get indexes of rows with year before 1974
indexYear = rtmrain[rtmrain['year'] < 1974].index
 
# Delete these row indexes from dataFrame
rtmrain.drop(indexYear , inplace=True)

# reseting the index:
rtmrain = rtmrain.reset_index()
rtmrain.drop('index',1,inplace=True)

null_cols = rtmrain.isna().sum()
null_cols[null_cols > 0] # only two blanks now

#first index we change to 76 as it has a similar rainy pattern as another day
#second index is zero all round
ind_nan = rtmrain[(rtmrain['DR'].isna() == True)].index
rtmrain['DR'][ind_nan[0]] = 76
rtmrain['DR'][ind_nan[1]] = 0
null_cols = rtmrain.isna().sum()
null_cols[null_cols > 0] # zero blanks now
# computer doesnt seem to like my way of changing the data, its giving me a warning, 
# but I tried different ways with the same result, and it seems to work anyway.

# DR - duration of precipitation (in 0.1 hours), I want to change this to real hours and minutes:
rtmrain['Total precip in hours'] = rtmrain['DR']/10
rtmrain['Total precip in min'] = rtmrain['Total precip in hours']*60

# columns RH & RHX use "-1" to indicate less than 0.5 mm , I prefer 0.25, as otherwise totals will be inconsistent
rtmrain['RH'] = rtmrain['RH'].replace(-1,0.25)
rtmrain['RHX'] = rtmrain['RHX'].replace(-1,0.25)

#changing column names to make them easier to read
rtmrain = rtmrain.rename(columns={'YYYYMMDD': 'Date', 'RH': 'total_precip/day', 'RHX': 'max_preciphour'})

# looking at outliers for possible mistakes in data
q = rtmrain["DR"].quantile(0.99)
rtmrain[rtmrain["DR"] > q].sort_values(by = 'DR')

q = rtmrain["total_precip/day"].quantile(0.99)
rtmrain[rtmrain["total_precip/day"] > q].sort_values(by = 'total_precip/day')

q = rtmrain["max_preciphour"].quantile(0.99)
rtmrain[rtmrain["max_preciphour"] > q].sort_values(by = 'max_preciphour')

# I think it looks OK, only date 1975-06-23 looks a bit out there, but according to online resources,
# there was a strange snow-spell in UK, in these dates in June?
# https://www.birminghammail.co.uk/news/midlands-news/snow-scenes-june-1975---9375720
# https://www.netweather.tv/forum/topic/29533-june-1975-snow-in-june/
# https://www.theweatheroutlook.com/twoother/twocontent.aspx?type=tystat&id=760

## 3 Creating a new clasification
Rain measured in mm can be a bit abstract, one millimeter of rainfall is the equivalent of one liter of water per square meter. Yet how much is a lot of rain? I found the following clasification from wikipedia and will add it to my data set:

Light rain: < 2.5 mm per hour

Moderate rain: between 2.5 mm to 7.6 mm per hour

Heavy rain: between 7.6 mm to 50 mm per hour

Violent rain: > 50 mm per hour

clasification from https://en.wikipedia.org/wiki/Rain#Measurement


In [ ]:
# function to identify the maximum precipitation recorded in one hour and add classification
def raintype(s):
    if s['max_preciphour'] == 0:
        return '0 dry'
    elif (s['max_preciphour'] > 0) & (s['max_preciphour'] <= 2.5):
        return '1 light'
    elif (s['max_preciphour'] > 2.5) & (s['max_preciphour'] <= 7.6):
        return '2 moderate'
    elif (s['max_preciphour'] > 7.6) & (s['max_preciphour'] <= 50):
        return '3 heavy'
    else:
        return '4 violent'
    
rtmrain['raintype'] = rtmrain.apply(raintype, axis=1)

rtmrain.head()

## 4 Total rainy days per year
The first thing I was curious about was the amount of rainy days per year and weather this amount has increased or decreased over the years

In [ ]:
# counting total rainy days per year 
# removing 2020 as the year is not complete
wetdays = rtmrain[(rtmrain['raintype'] != '0 dry') & (rtmrain['year'] != 2020)]
wetdays = wetdays.groupby(['year'])['raintype'].count()

#plotting
plt.figure(figsize=(20, 8))
plt.title('Rainy days per year')

x = np.arange(0, len(wetdays)) # amount of years to plot
xrange = np.arange(0, len(wetdays), 2) # for labeling

coef = np.polyfit(x, wetdays, 1)
poly1d_fn = np.poly1d(coef)

plt.plot(x, wetdays)
plt.plot(x, poly1d_fn(x), ls=":")

plt.xticks(xrange, np.arange(1974, 2020,2))

plt.xlabel('Year')
plt.ylabel('Total rainy days')
plt.savefig('../images/1totalrainydays.png')
plt.show()


The amount of rainy days per year ranges between 200 and 260. It appears to be a bit cyclical with 2 to 3 years of rain and then a drier year with around 50 less days of rain. In general the trend seems to be in decline, with the last few years having less rainy days than 50 years ago.

## 5 Amount of rain during a rainy day
So we are seing a decline on rainy days per year, but how much rain does each rainy day have, and how has this changed

In [ ]:
#removing 2020 as the year is not complete
rtmrainmean = rtmrain[(rtmrain['raintype'] != '0 dry') & (rtmrain['year'] != 2020)]

#calculating mean per year
rtmrainmean = rtmrainmean.groupby(['year'])['total_precip/day'].mean()

#plotting
plt.figure(figsize=(20, 8))
plt.title('Mean precipitation during rainy days per year')

x = np.arange(0, len(rtmrainmean)) # amount of years to plot
xrange = np.arange(0, len(rtmrainmean), 2) # for labeling

coef = np.polyfit(x, rtmrainmean, 1)
poly1d_fn = np.poly1d(coef)

plt.plot(x, rtmrainmean)
plt.plot(x, poly1d_fn(x), ls=":")


plt.xticks(xrange, np.arange(1974, 2020,2))

plt.xlabel('Year')
plt.ylabel('Avg precipitation in mm')
plt.savefig('../images/2meanrain.png')
plt.show()


So even if the amount of rainy days has decreased, we can see that in average the amount of rain during each rainy day has increased. 

According to the KNMI (The Royal Netherlands Meteorological Institute) this is due to climate change. They have observed data from all over the country in the last 100 years, and estimate an increase of 25% of precipitation. This increase, they have noted, is partialy due to the warmer summer months we are having, where the warm air causes more waterdamp in the air.

## 6 Rain per semester
As per the KNMI most of the increase in precipitation over the last few years is in the second half of the year, so I wanted to check if I could find the same in my data.

In [ ]:
# total per decade (show semesters)

# removing first and last decade as they are not 10 year info:
decades = rtmrain['decade'].unique()
decades = decades[1:-1]
stackedrain = rtmrain[rtmrain['decade'].isin(decades)]
stackedrain = stackedrain.pivot_table(index=['decade'],columns=['semester'],values=['total_precip/day'], aggfunc=sum)


ax = stackedrain.plot(kind='bar', stacked=True, figsize=(20, 10.5))
ax.set_title('Total precipitation per year')
ax.set_xlabel('Decade')
ax.set_ylabel('Precipitation in mm')
ax.set_label(stackedrain['total_precip/day'])
plt.legend((1,2,3,4), loc = "upper left",title="Semester")

# .patches is everything inside of the chart
for rect in ax.patches:
    # Find where everything is located
    height = rect.get_height()
    width = rect.get_width()
    x = rect.get_x()
    y = rect.get_y()

    # The width of the bar is the data value and can used as the label
    label_text = f'{height : .1f}'  # f'{height:.2f}' to format decimal values

    # ax.text(x, y, text)
    label_x = x + width  # adjust 0.2 to center the label
    label_y = y + height / 2
    ax.text(label_x, label_y, label_text, ha='right', va='bottom', fontsize=15)
plt.savefig('../images/3rainsemester.png')
plt.show()


In the last 4 decades we can observe that the amount of rain in the 1st semester of the year remains very stable, with even last decade 2010 having the same amount of precipitation that in the 80's (36.3 thousand mm of rain).

During the second half of the year, however, we note an increase  in precipitation of 7 thousand mm in the course of the 4 decades.

## 7 Total precipitation per month
I wanted to look closer at the data and try to identify and changes or trends per month for the last 4 decades

In [ ]:
raindecade = rtmrain[rtmrain['decade'].isin(decades)]
raindecade = raindecade.pivot_table(index=['month'],columns=['decade'],values=['total_precip/day'], aggfunc=sum)

plt.figure(figsize=(20, 8))
plt.title('Total precipitation per month (full decades)')
plt.plot(raindecade)
plt.xticks(np.arange(1, 13, 1))
plt.xlabel('Month')
plt.ylabel('Precipitation in mm')
plt.legend((decades), loc = "upper left")
plt.savefig('../images/4totalpermonth.png')
plt.show()


Its interesting to see that in the 80's, the month with most precipitation was October, in the 90's it was September and in 2000 and 2010 it is August. It seems every decade brought the peak a month earlier.
From April to June, the amount of precipitation seems very similar for all 4 decades, but its in the summer months (July and August specifically) where we see the biggest differece, where the 2 oldest decades remain low, and the most recent decade go up in precipitation,leaving a glaring gap of difference.

## 8 Rain type patterns
We have observed so far that its not the rainy days that have increased over the years, but rather the amount of rain that falls when it does rain. With this plot I wanted see what rain clasifications have increased over the years.

In [ ]:
# removing first and last half_decade as they are not 5 year info:
half_decades = rtmrain['half_decade'].unique()
half_decades = half_decades[1:-1]

rtmraintype = rtmrain[rtmrain['half_decade'].isin(half_decades)]

#selecting only heavy rain
rtmraintype = rtmraintype[rtmraintype['raintype'].isin(['1 light','2 moderate', '3 heavy', '4 violent'])]
rtmraintype = rtmraintype.pivot_table(index=['half_decade'],columns=['raintype'],values=['max_preciphour'], aggfunc=len)

ax = rtmraintype.plot(kind='bar', stacked=True, figsize=(20, 10.5))

ax.set_title('Type of rain per Quinquennial')
ax.set_xlabel('Quinquennials (5 years)')
ax.set_ylabel('Rainy days')
plt.legend(('light <2.5mm','moderate 2.5-7.6mm', 'heavy 7.6-50mm', 'violent >50mm'), loc = "lower left",title="rain type")
for rect in ax.patches:
    # Find where everything is located
    height = rect.get_height()
    width = rect.get_width()
    x = rect.get_x()
    y = rect.get_y()

    # The width of the bar is the data value and can used as the label
    label_text = f'{height}'  # f'{height:.2f}' to format decimal values

    # ax.text(x, y, text)
    label_x = x + width  # adjust 0.2 to center the label
    label_y = y + height / 2
    ax.text(label_x, label_y, label_text, ha='right', va='bottom', fontsize=15)

plt.savefig('../images/5typeofrain.png')
plt.show()


The trend shows that while light rain days have been decreasing, heavy and violent rain days have increased

## 9 Rain profile for RTM, based on the last 5 years
Finally, based on the precipitation of the last 5 years, I wanted to create a profile to know what I could expect in the comming months for rain in Rotterdam

In [ ]:
# checking last 5 years
raintype_quarter = rtmrain[rtmrain['year'].isin(['2019', '2018', '2017', '2016','2015'])]

raintype_quarter = raintype_quarter.pivot_table(index=['quarter'],columns=['raintype'],values=['max_preciphour'], aggfunc=len)

raintype_quarter = raintype_quarter/5

ax = raintype_quarter.plot(kind='bar', stacked=True, figsize=(20, 10.5))

ax.set_title('Raintype per quarter')
ax.set_xlabel('quarter')
ax.set_ylabel('count')
plt.legend(('dry','light <2.5mm','moderate 2.5-7.6mm', 'heavy 7.6-50mm', 'violent >50mm'), loc = "lower left",title="rain type")
for rect in ax.patches:
    # Find where everything is located
    height = rect.get_height()
    width = rect.get_width()
    x = rect.get_x()
    y = rect.get_y()

    # The width of the bar is the data value and can used as the label
    label_text = f'{height:.1f}'  # f'{height:.2f}' to format decimal values

    # ax.text(x, y, text)
    label_x = x + width  # adjust 0.2 to center the label
    label_y = y + height / 2
    ax.text(label_x, label_y, label_text, ha='right', va='bottom', fontsize=15)

plt.savefig('../images/6raintypequarter.png')
plt.show()


The first and last quarter of the year seem to have the most heavy rainfalls, but the summer has the most violent rainfalls of the year. Our current quarter (1 April - June) seems to be the best quarter of the year with the most dry days and most light rain days. 

In the plot below we also see that during the first and last quarter of the year we can expect rainfalls to last around 3 hours (with some terrible exceptions of over 20 hours), while the 2 middle quarters have a smaller average of around 1 hour per rain for each precipitation.

In [ ]:
#removing 2020 as the year is not complete
durationquarter = rtmrain[rtmrain['raintype'] != '0 dry']

durationquarter = durationquarter[durationquarter['year'].isin(['2019', '2018', '2017', '2016','2015'])]

plt.figure(figsize=(20, 10))
g = sns.boxplot(x='quarter', y='Total precip in hours', data=durationquarter)
g.set_xticklabels(g.get_xticklabels(), rotation=90)

g.set_xlabel("quarter")
g.set_ylabel("hours of rain")
g.set_title('Raining hours range')
plt.savefig('../images/7rainingrange.png')
plt.show()


## 10 Conclusion
Rain patterns in RTM have been shifting from 1974 to today. We notice less rainy days, but harder rainfalls when it does rain. Notabily also an increase in violent rainfalls (which means more than 50 lites of water per square meter during an hour). 

We notice that the first half of the year remains quite stable over the decades, but its the second half, specifically around the summer months where the rain patterns are shifting. According to the KNMI this is due to the increase in temperature. Warmer summer months increase the waterdamp in the air and create more rainfalls.

The KNMI has created 6 predictions for the future, but in all of them they expect the amount of violent rainfalls to keep increasing and amount of days with rain to decrease, specially in the summer.

Its hard to say what effect this will have on our planet, but my grandfather will definitely not be happy as heavy rainfalls ruin his garden, and light rains are the best for his vegetables!!